## 009.Using Simphy to simulate trees


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sys, subprocess, time, dendropy, os, copy
import numpy as np
from sklearn import manifold, metrics, cluster, neighbors, decomposition, preprocessing
import treesignal

In [2]:
def read_tree(path):
    tree = dendropy.Tree.get(path = path, schema="newick", preserve_underscores=True)
    for leaf in tree.leaf_nodes(): # simphy default is to name as numbers
        leaf.taxon.label = "s" + leaf.taxon.label # 1_2_1 becomes s1_2_1
    return tree

def simphy(ntaxa=8, n_sp=4, nloci=2):
    ## limited to less than 1000 species trees and loci per sptree (just b/c of padding)
    tmp_dir = "/tmp/simphy"
    cli = "/home/leo/Academic/Biomath/Simulation/SimPhy/bin/simphy"
    # sb, ld, lb, lt are rate of transfers, births, etc. sp. su, st are pop size, tree height, subst rate
    cli += " -sb f:0.0000001 -ld f:0.0000005 -lb f:0.00000051 -lt f:0.00000005 -sp f:10000 -su f:0.0001 -st f:1000000"
    # sg is generation time, ls is minimum # of species per locus tree
    cli += " -sg f:1 -sl F:" + str(ntaxa) + " -ls 4 -o " + tmp_dir
    cli += " -rs " + str(n_sp) + " -rl F:" + str(nloci) + " "
    now_time = time.perf_counter()
    proc_run = subprocess.run(cli, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    now_time = time.perf_counter() - now_time
    print("simphy finished in " + str(now_time) + "secs")
    
    ## read g_treesXX.trees and s_tree.treesfiles from directories 1 ... n_sptrees
    ## (where XX is left-padded number of loci)
    
    if nloci < 10:    n_pad=1
    elif nloci < 100: n_pad=2
    else:             n_pad=3
    if n_sp < 10:    s_pad=1
    elif n_sp < 100: s_pad=2
    else:            s_pad=3
    sp_trees = list(range(n_sp)) # we know sizes beforehand, just fill with arbitrary numbers
    gn_trees = list(range(n_sp))
    for i in range(n_sp):
        spath = tmp_dir + "/" + str(i+1).zfill(s_pad) # this path will be used below
        sp_trees[i] = read_tree(spath + "/s_tree.trees") # dendropy read plus name change
        gn_trees[i] = list(range(nloci))
        for j in range(nloci): 
            gn_trees[i][j] = read_tree(spath + "/g_trees" + str(j+1).zfill(n_pad) + ".trees")
    proc_run = subprocess.run("rm -rf " + tmp_dir, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    return sp_trees, gn_trees   

In [3]:
st, gt = simphy(ntaxa=20, nloci=20, n_sp=5)
spstring = ""
for sptree in st:
    spstring += sptree.as_string("newick",suppress_edge_lengths=True).rstrip()
print ("spstring:: ", spstring) # dendropy.TreeList(st) also returns treelist object
for row in gt:
    for tre in row:
        print (tre.as_string("newick",suppress_edge_lengths=True))

simphy finished in 0.02657169300073292secs
spstring::  (s1,((((s15,((s10,(s16,s3)),s19)),s13),(((s20,(s2,s14)),(((s17,s6),(s12,(s9,s8))),s11)),(s7,s5))),(s18,s4)));((((s5,s8),(s9,((s6,s3),(s7,s10)))),(s2,(s1,s4))),((s18,((s14,(s15,(s20,s19))),(s13,s11))),((s17,s12),s16)));(((((s16,s4),(s5,s10)),((((s11,s14),((s8,s19),s7)),(s17,(s18,s6))),((s2,s12),s1))),(s9,(s15,(s3,s13)))),s20);(((s13,(s10,s12)),((((s8,s9),(s1,s5)),s7),(s6,(((s11,s2),s14),(s4,s3))))),(((s19,s18),(s17,s20)),(s15,s16)));(((((s3,s11),(s5,s10)),((s6,(s1,s9)),(s4,(s7,s2)))),s8),((s14,(s20,(s19,s13))),(((s12,s15),s17),(s18,s16))));
('s1_0_0',(('s18_0_0','s4_0_0'),((((((('s9_0_0','s8_0_0'),'s12_0_0'),(('s17_0_0','s6_0_0'),(('s17_10_0','s6_10_0'),('s17_9_0','s6_9_0')))),'s11_0_0'),('s14_0_0','s2_0_0')),'s7_0_0'),((((('s10_1_0',('s3_1_0',('s16_1_0',('s16_5_0','s16_4_0')))),('s19_3_0',(('s3_3_0','s16_3_0'),'s10_3_0'))),'s19_1_0'),('s15_2_0','s15_1_0')),((('s5_0_0','s5_11_0'),('s13_0_0',('s13_7_0','s13_8_0'))),('s3_0_0',('s19_0_

In [5]:
sp2 = treesignal.lowlevel_randomise_trees_with_spr_string(spstring, n_copies=20, n_spr=2)

In [6]:
ts = treesignal.TreeSignal(sp_trees = dendropy.TreeList.get( data=sp2, schema="newick"), replicates=500)

In [ ]:
feat_mat = []
label = []
idx = 0
for row in gt:
    for tre in row:
        spectrum = ts(tre)
        if spectrum.max() > -1.:
            feat_mat.append(spectrum)
            label.append(idx)
    idx+=1
feat_mat = np.array(feat_mat)
print ("dimensions: ", feat_mat.shape, len(label)); # print (feat_mat)

NameError: name 'gt' is not defined

In [ ]:
#print (feat_mat.max(1)); # print (ts.sp_string)
transf=manifold.MDS(n_components=2).fit_transform(feat_mat[:,:180]) ## TSNE or MDS

In [ ]:
#  %%script false # tell jupyter NOT TO RUN cell (on linux only)
fig, axes = plt.subplots(1) ; fig.set_size_inches(10, 6)
fig.subplots_adjust(top=.99, bottom=.01, left=.02, right=.98, wspace=.1, hspace=.2)
jit = transf.max() * np.random.normal(scale=1e-4,size=feat_mat.shape[0]) # avoid complete overlap of points
axes.scatter(transf[:,1]+jit, transf[:,0]+jit[::-1], c=label, edgecolor="black", cmap="plasma", alpha=.8, s=80)
axes.set_title("MDS",  fontsize=18)